In [577]:
# Install missing package

import sys
# !conda install --yes --prefix {sys.prefix} requests
# !conda install --yes --prefix {sys.prefix} beautifulsoup4
# !conda install --yes --prefix {sys.prefix} pandas
# !conda install --yes --prefix {sys.prefix} geopandas
# !conda install --yes --prefix {sys.prefix} -c conda-forge folium

In [578]:
# Module importation

import json
import requests
from bs4 import BeautifulSoup
import re

import os.path

import pandas as pd
import numpy as np

import geopandas
import folium 
from folium.plugins import BeautifyIcon

from shapely.geometry import Point
from geopandas import GeoDataFrame

from IPython.display import display, HTML

import colorsys

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 4000)
pd.set_option('max_colwidth', 500)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)

# Prevent dataframe html class and td tag to wrap content
display(HTML("<style>.dataframe td th {white-space: nowrap;}</style>"))

def pretty_print(df):
    html_to_display = df.to_html()
    return display(HTML(html_to_display))

In [579]:
# Global variables definition
url = 'http://badiste.fr'
data_filename = 'data.json'
update_data = False

def HSVToRGB(h, s, v):
    r, g, b = colorsys.hsv_to_rgb(h, s, v) 
    return (int(255*r), int(255*g), int(255*b)) 
 
def getDistinctColors(n): 
    huePartition = 1.0 / (n + 1) 
    return (HSVToRGB(huePartition * value, 1.0, 1.0) for value in range(0, n)) 

def rgb_to_hex(r, g, b):
    return f'{r:02x}{g:02x}{b:02x}'


In [580]:
if update_data or not(os.path.isfile(data_filename)):
    
    # Get website main page
    main_page = requests.get(url)

    soup = BeautifulSoup(main_page.content, 'html.parser')
    tournament_menu = soup.find(id="m_tournoi")

    # Navigate to tournament list page
    tournament_list_page = requests.get(f"{url}/{tournament_menu.get('href')}")

    soup = BeautifulSoup(tournament_list_page.content, 'html.parser')

    tournaments_info = []
    for idx, each_tournament in enumerate(soup.find_all('a', {'href': re.compile(r'^tournoi.*')})):
        each_tournament_page = requests.get(f"{url}/{each_tournament.get('href')}")

        soup = BeautifulSoup(each_tournament_page.content, 'html.parser')
        
        table = soup.find('table', class_='formulaire', attrs={'summary':'description du tournoi'})
        
        # Ignore page that are not related to a tournament
        if table is None:
            continue

        tournament_info = {}    
        
        # Extract every key/value pairs from table and store them in Json
        # Only extract 2 columns pattern
        rows = table.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            if(len(cols)==2): 
                key = cols[0].get_text("\n")
                value = cols[1].get_text("\n")
                tournament_info[key] = value
        
        # Extract GPS coordinates from location information
        try:
            coordinates = re.search(r'GPS : [-0-9.]*, [-0-9.]*', tournament_info['Lieu']).group(0)
            lat, lon = [float(i) for i in coordinates[6:].split(", ")]
            tournament_info['latitude'], tournament_info['longitude'] = lat, lon
        except AttributeError:
            tournament_info['latitude'], tournament_info['longitude'] = np.nan, np.nan
        
        tournaments_info.append(tournament_info)
    
    # Save in file
    with open(data_filename, 'w', encoding='utf8') as file:    
        json.dump(tournaments_info, file, indent=4, ensure_ascii=False)

In [581]:
with open(data_filename, 'r', encoding='utf8') as file:
        tournaments_info = json.load(file)

df = pd.DataFrame.from_dict(tournaments_info)

# # Drop row with nan value
# df = df.dropna()
df = df.replace('non spécifié', np.nan)

# Drop useless columns
df = df.drop(columns=["Horaires de convocations", "Documents"])

# Normalize "Frais d'inscription" values
df["Frais d'inscription"] = df["Frais d'inscription"].replace(
    r'[\w]* tableau[\w]* :|\n| +$|^ +', '', regex=True)

df["Frais d'inscription"] = df["Frais d'inscription"].replace({r'€ ': r'€ - '}, regex=True)

# Extract zipcodes from "Lieu"
df["zipcode"] = df["Lieu"].str.extract(r'\D*(\d{5})\D*')
df = df.drop(columns=["Lieu"])

# Normalize "Catégorie" values
df["Catégorie"] = df["Catégorie"].replace({r'[^JSV]': ''},
                                          regex=True)

df["Catégorie"] = df["Catégorie"].replace({r'(?<=\w)(?=\w)': r' - ',
                                           r'J': 'Jeune', 
                                           r'S': 'Senior',
                                           r'V': 'Veteran'},
                                          regex=True)              

def normalize_ranking(rank: str):
    pass

# df["Classements autorisés"] = df["Classements autorisés"].map(lambda x: f'test {x}')

display(df["Classements autorisés"].unique())




array(['NC/P12/P11/P10/D9/D8/D7/R6/R5/R4/N3', 'D7àNC',
       'R5,R6,D7,D8,D9,P10,P11,P12,NC', 'R,D,P(0à600points)', 'R,D,P',
       'N2,N3,R,D,P', 'N2,N3,R,D,P,NC', 'N3/R4,R5/R6,D7/D8,D9/P',
       'N,R,D,P', 'N2,R,D,P', 'R5àNC', 'R,D,P,NC', 'N,R,D,P,NC',
       'N1,N2,N3,R4,R5,R6,D7,D8,D9', 'N1,N2,N3,R4,R5', 'Be,Ca',
       'N3,R,D,P,Vet', 'Po,Be,Mi,Ca,Ju', 'po,be,mi,ca,ju', 'N3/NC',
       'D,P,NC', 'Mb,Po,Be,Mi,Ca,Ju', 'N3,R,D,P', 'R6-D7,D8-D9,P10-NC',
       'N2àNC', 'N1/N2/N3/R4/R5/R6/D7/D8/D9/P/NC', 'N3,R,D,P,NC',
       'R,D,P,Nc', 'N-R-D', 'D9/P12', 'SérieElite:1000et+Série1:4',
       'N1,N2,N3,R,D,P', 'N2/N3/R4/R5/R6/D7/D8/D9/P/NC', 'N3/D8',
       'N,R,D;P', 'N2,R,D,P,NC', 'NC,P12,P11,P10,D9',
       'N2,N3,R4,R5,R6,D7,D8,D9,P10,P11,P12',
       'R5/R6/D7/D8/D9/P10/P11/P12/NC',
       'N3,R4,R5,R6,D7,D8,D9,P10,P11,P12', 'NC,P,D9', 'R/P', 'N1-D8',
       'N3,R5,R7,D9,P', 'N2àP12(NC)', 'R5,R6,D7,D8,D9,P10,P11,P12',
       'R6/NC', 'N1,N2,N3,R4,R5,R6,D7,D8,D9,P,NC', 'R,D,P,Po,

In [582]:
# Create GeoDataFrame from pandas dataframe
gdf = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df["longitude"], df["latitude"]), 
                             crs="EPSG:4326")

gdf = gdf.drop(columns = ["latitude", "longitude"])

tournament_types = gdf["Type de tournoi"].unique()

gdf_by_tournament_type_list = [gdf.loc[gdf['Type de tournoi'] == tournament_type]
               for tournament_type in tournament_types]

for gdf_by_tournament_type in gdf_by_tournament_type_list:
    display(gdf_by_tournament_type.head(3))

,Club organisateur,Date,Durée du tournoi,Date limite d'inscription sur badiste,Date limite d'inscription réelle,Type de tournoi,Catégorie,Classements autorisés,Frais d'inscription,zipcode,geometry
0,Associat. Badminton Pithiviers (ABP45),Vendredi 25 mars 2022,2 jour(s),Samedi 15 janvier 2022,Mardi 15 mars 2022,National,Senior - Veteran,NC/P12/P11/P10/D9/D8/D7/R6/R5/R4/N3,13€,45300,POINT (2.25468 48.18390)
2,Massy Athletic Sports (MAS91),Samedi 26 mars 2022,2 jour(s),Jeudi 3 fevrier 2022,Samedi 26 fevrier 2022,National,Senior,"R5,R6,D7,D8,D9,P10,P11,P12,NC",16€ - 21€,91300,POINT (2.25316 48.73550)
3,Union Sportive Saint Arnoult (USSA78),Samedi 26 mars 2022,2 jour(s),Mardi 15 fevrier 2022,Vendredi 25 fevrier 2022,National,Senior,"R,D,P(0à600points)",14€ - 20€,78730,POINT (1.95398 48.57420)


,Club organisateur,Date,Durée du tournoi,Date limite d'inscription sur badiste,Date limite d'inscription réelle,Type de tournoi,Catégorie,Classements autorisés,Frais d'inscription,zipcode,geometry
1,Les Fous Du Volant Guipry-Messac-Pipriac (USGM35),Vendredi 25 mars 2022,1 jour(s),Mardi 22 mars 2022,Mardi 22 mars 2022,International,Senior,D7àNC,11€,35480,POINT (-1.82838 47.80850)
5,Aix Universite Club Badminton (AUCB13),Samedi 26 mars 2022,2 jour(s),Vendredi 25 fevrier 2022,Lundi 7 mars 2022,International,Senior,"N2,N3,R,D,P",18€ - 21€,13090,POINT (5.41279 43.51730)
8,Amicale Laïque Hericourt (ALH70),Samedi 26 mars 2022,2 jour(s),Mardi 1 mars 2022,Samedi 5 mars 2022,International,Veteran,"N,R,D,P",10€ - 14€ - 18€,70400,POINT (6.75892 47.57520)


,Club organisateur,Date,Durée du tournoi,Date limite d'inscription sur badiste,Date limite d'inscription réelle,Type de tournoi,Catégorie,Classements autorisés,Frais d'inscription,zipcode,geometry
15,Comité 83 (CD83),Samedi 26 mars 2022,2 jour(s),Vendredi 11 mars 2022,Lundi 14 mars 2022,Départemental,Jeune - Veteran,"N,R,D,P",12€ - 15€ - 15€,83170,POINT (6.05818 43.41110)
24,Comité Départemental 21 (CD21),Samedi 26 mars 2022,2 jour(s),Mercredi 16 mars 2022,Lundi 14 mars 2022,Départemental,Senior,"N,R,D,P",10€ - 15€,21600,POINT (5.06090 47.29630)
28,Comité Départemental 37 (CD37),Samedi 26 mars 2022,2 jour(s),Vendredi 18 mars 2022,Vendredi 18 mars 2022,Départemental,Jeune,"Po,Be,Mi,Ca,Ju",7€,37110,POINT (0.91778 47.59640)


,Club organisateur,Date,Durée du tournoi,Date limite d'inscription sur badiste,Date limite d'inscription réelle,Type de tournoi,Catégorie,Classements autorisés,Frais d'inscription,zipcode,geometry
16,Beaurepaire Badminton (BBAD38),Samedi 26 mars 2022,2 jour(s),Vendredi 11 mars 2022,Vendredi 11 mars 2022,Régional,Jeune - Veteran - Senior,"R,D,P",17€ - 22€,38270,POINT (5.06335 45.33890)
25,Comité Départemental 06 (CD06),Samedi 26 mars 2022,1 jour(s),Jeudi 17 mars 2022,Jeudi 17 mars 2022,Régional,Jeune,"Be,Ca",11€ - 11€,06130,POINT (6.91082 43.63720)
27,Badminton Club Serignan (BCS34),Samedi 26 mars 2022,2 jour(s),Vendredi 18 mars 2022,Vendredi 18 mars 2022,Régional,Jeune - Senior,"R,D,P",15€ - 17€,34410,POINT (3.28690 43.26580)


In [583]:
# Create a Map instance
m = folium.Map(location=[45, 5], tiles='OpenStreetMap', zoom_start=5)

disctinct_colors = list(getDistinctColors(len(gdf_by_tournament_type_list)))
disctinct_colors = [f'#{rgb_to_hex(*i)}' for i in disctinct_colors]

for idx, gdf_by_tournament_type in enumerate(gdf_by_tournament_type_list):
    icon = folium.Icon(color='cadetblue', icon_color=disctinct_colors[idx], icon='star')

    marker = folium.Marker(icon=icon, tooltip="esfds")

    fields = ["Club organisateur", "Date", 
              "Date limite d'inscription réelle", "Type de tournoi", 
              "Catégorie", "Classements autorisés", "Frais d'inscription", "zipcode"]
    
    tooltip = folium.GeoJsonTooltip(fields=fields)

    gdf_by_tournament_type = gdf_by_tournament_type.dropna()

    folium.features.GeoJson(gdf_by_tournament_type[gdf_by_tournament_type['geometry'].is_valid],
                            name=gdf_by_tournament_type.iloc[1]["Type de tournoi"],
                            tooltip=tooltip,
                            marker=marker).add_to(m)

# Add Layer Control to pick feature to display
folium.LayerControl(collapsed=True).add_to(m)

In [584]:
# Generate html map file
m.save('map.html')

#Show map
m

